# Camada Gold
Essa etapa contém dados agregados e otimizados para consultas e consumo por ferramentas de BI.

## Objetivos:
- Efetuei a leitura dos dados da tabela Delta da Camada Silver.

- Criei tabelas agregadas por período (ano/mês), distrito e batalhão.

- As tabelas Gold criadas estão no formato Delta e são gerenciadas pelo Unity Catalog.

## Decisões Técnicas:
- Novamente tentei me ater ao paralelismo, por isso as operações como _GroupBy_ (agrupamento) e _Agg_ (agregação) são executadas de forma paralela pelo Spark.

-  As agregações são calculadas e armazenadas previamente em tabelas separadas.

- Optei por utilizar _"overwrite"_ na escrita para reescrever os dados com os valores mais recentes das camadas anteriores.



In [0]:
# Importando as bibliotecas necessárias para agregações
from pyspark.sql.functions import col, count, sum, year, month, dayofmonth, to_date, lit, current_timestamp
from pyspark.sql.types import LongType, DoubleType, StringType, DateType

# Tabela Silver como fonte
silver_table_name_full = "`workspace`.`default`.`fire_incidents_silver`"
silver_table_name_simple = "fire_incidents_silver"

# Tabelas Gold como destino
gold_agg_by_time_full = "`workspace`.`default`.`fire_incidents_gold_by_time`"
gold_agg_by_time_simple = "fire_incidents_gold_by_time"

gold_agg_by_district_full = "`workspace`.`default`.`fire_incidents_gold_by_district`"
gold_agg_by_district_simple = "fire_incidents_gold_by_district"

gold_agg_by_battalion_full = "`workspace`.`default`.`fire_incidents_gold_by_battalion`"
gold_agg_by_battalion_simple = "fire_incidents_gold_by_battalion"

print(f"Fonte de dados (Camada Silver): {silver_table_name_full}")
print(f"Destino de dados (Gold - Por Tempo): {gold_agg_by_time_full}")
print(f"Destino de dados (Gold - Por Distrito): {gold_agg_by_district_full}")
print(f"Destino de dados (Gold - Por Batalhão): {gold_agg_by_battalion_full}")

In [0]:
# Lendo os dados da camada Silver
print(f"\nLendo dados da Camada Silver ({silver_table_name_full})...")

silver_df = spark.table(silver_table_name_simple)

print("Schema da Camada Silver (esperado com tipos corretos e nulos tratados):")
silver_df.printSchema()
print(f"Total de registros lidos da Camada Silver: {silver_df.count()}")

In [0]:
# Agregando por Ano/Mês
print("\nCriando agregação Gold por Tempo (Ano/Mês)...")

silver_df_cleaned_dates = silver_df.withColumn("Incident Date", col("Incident Date").cast(DateType()))

gold_by_time_df = silver_df_cleaned_dates.groupBy(year(col("Incident Date")).alias("incident_year"),
                                                  month(col("Incident Date")).alias("incident_month")) \
                                           .agg(count("ID").alias("total_incidents"), # Contagem de incidentes únicos
                                                sum("Estimated Property Loss").alias("total_property_loss"), # Soma de perdas
                                                sum("Estimated Contents Loss").alias("total_contents_loss")) \
                                           .withColumn("gold_loaded_at", current_timestamp()) # Metadado de carga

print("\nSchema da Tabela Gold (Por Tempo):")
gold_by_time_df.printSchema()
print(f"Total de registros na Gold (Por Tempo): {gold_by_time_df.count()}")

# Salvando a tabela Gold por Tempo
gold_by_time_df.write \
               .format("delta") \
               .mode("overwrite") \
               .option("overwriteSchema", "true") \
               .option("delta.columnMapping.mode", "name") \
               .saveAsTable(gold_agg_by_time_simple)
print(f"Tabela Gold '{gold_agg_by_time_full}' salva com sucesso.")

# Agregando por Distrito
print("\nCriando agregação Gold por Distrito...")

gold_by_district_df = silver_df.groupBy(col("neighborhood_district").alias("district_name")) \
                                 .agg(count("ID").alias("total_incidents"),
                                      sum("Estimated Property Loss").alias("total_property_loss"),
                                      sum("Estimated Contents Loss").alias("total_contents_loss")) \
                                 .withColumn("gold_loaded_at", current_timestamp())

print("\nSchema da Tabela Gold (Por Distrito):")
gold_by_district_df.printSchema()
print(f"Total de registros na Gold (Por Distrito): {gold_by_district_df.count()}")

# Salvando a tabela Gold por Distrito
gold_by_district_df.write \
                   .format("delta") \
                   .mode("overwrite") \
                   .option("overwriteSchema", "true") \
                   .option("delta.columnMapping.mode", "name") \
                   .saveAsTable(gold_agg_by_district_simple)
print(f"Tabela Gold '{gold_agg_by_district_full}' salva com sucesso.")

# Agregando por Batalhão
print("\nCriando agregação Gold por Batalhão...")

gold_by_battalion_df = silver_df.groupBy(col("Battalion").alias("battalion_id")) \
                                 .agg(count("ID").alias("total_incidents"),
                                      sum("Estimated Property Loss").alias("total_property_loss"),
                                      sum("Estimated Contents Loss").alias("total_contents_loss")) \
                                 .withColumn("gold_loaded_at", current_timestamp())

print("\nSchema da Tabela Gold (Por Batalhão):")
gold_by_battalion_df.printSchema()
print(f"Total de registros na Gold (Por Batalhão): {gold_by_battalion_df.count()}")

# Salvando a tabela Gold por Batalhão
gold_by_battalion_df.write \
                    .format("delta") \
                    .mode("overwrite") \
                    .option("overwriteSchema", "true") \
                    .option("delta.columnMapping.mode", "name") \
                    .saveAsTable(gold_agg_by_battalion_simple)
print(f"Tabela Gold '{gold_agg_by_battalion_full}' salva com sucesso.")

print("\nAgregações da Camada Gold concluídas com sucesso!")

In [0]:
# Verificação Final das Tabelas Gold
print("\n--- Verificando a Tabela Gold: Por Tempo (Ano/Mês) ---")
final_gold_by_time_df = spark.table(gold_agg_by_time_simple)
display(final_gold_by_time_df.orderBy("incident_year", "incident_month").limit(10))

print("\n--- Verificando a Tabela Gold: Por Distrito ---")
final_gold_by_district_df = spark.table(gold_agg_by_district_simple)
display(final_gold_by_district_df.orderBy(col("total_incidents").desc()).limit(10))

print("\n--- Verificando a Tabela Gold: Por Batalhão ---")
final_gold_by_battalion_df = spark.table(gold_agg_by_battalion_simple)
display(final_gold_by_battalion_df.orderBy(col("total_incidents").desc()).limit(10))

print("As tabelas agregadas estão prontas para consumo.")